# Student Loan Risk with Deep Learning

In [19]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [20]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [21]:
# Review the data types associated with the columns
loans_df.dtypes

,0
payment_history,float64
location_parameter,float64
stem_degree_score,float64
gpa_ranking,float64
alumni_success,float64
study_major_code,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64


In [22]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [23]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]



,credit_ranking
0,0
1,0
2,0
3,1
4,0


In [24]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [25]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [26]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Preview scaled dataset
X_train_scaled[:5]


array([[-0.56794375, -1.18088385, -0.19241296, -0.17192598, -0.45737585,
         1.86853316,  0.7479316 , -0.45120802,  0.73249574, -0.22553894,
        -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, -0.32236513, -0.0699402 ,
        -0.45384715, -0.08558383,  0.44779905, -0.03016065, -0.75844691,
        -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021,  0.05373275, -0.13451281,
         1.48146978,  0.80967349,  1.49315611, -1.36480934,  0.42579302,
         0.05358797],
       [ 0.21400374,  1.30301776, -0.24280683, -0.17192598, -0.50042426,
        -0.64737885, -0.11645477, -0.23168303, -1.36480934, -0.64002292,
        -0.97146166],
       [-0.00940983,  1.74956187, -1.30107808, -0.24714555, -0.37127904,
        -1.03444223, -0.98084114, -0.02261162,  0.03339405, -0.46238693,
         0.51951962]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [27]:
# Define the the number of inputs (features) to the model
input_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {input_features}")

Number of input features: 11


In [28]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = input_features * 2
print(f"Number of hidden nodes in the first hidden layer: {hidden_nodes_layer1}")

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = input_features
print(f"Number of hidden nodes in the second hidden layer: {hidden_nodes_layer2}")

# Define the number of neurons in the output layer
output_neurons = len(y.unique())
print(f"Number of neurons in the output layer: {output_neurons}")

Number of hidden nodes in the first hidden layer: 22
Number of hidden nodes in the second hidden layer: 11
Number of neurons in the output layer: 2


In [29]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation="softmax"))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
# Display the Sequential model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 22)                  │             264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 11)                  │             253 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │              24 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 541 (2.11 KB)

 Trainable params: 541 (2.11 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [31]:
# Compile the Sequential model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Before fitting the model, convert y_train to one-hot encoded format:
y_train_encoded = tf.keras.utils.to_categorical(y_train)
y_test_encoded = tf.keras.utils.to_categorical(y_test)

# Scale X test data as well
X_test_scaled = scaler.fit_transform(X_test)

# Fit the model using 50 epochs and the training data
history = model.fit(
    X_train_scaled,
    y_train_encoded,
    epochs=50,
    batch_size=32,
    verbose=1
    )


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5569 - loss: 0.6950   
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7292 - loss: 0.6042 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7281 - loss: 0.5525
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7161 - loss: 0.5547 
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7503 - loss: 0.5147 
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7474 - loss: 0.5170 
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7454 - loss: 0.5162 
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7490 - loss: 0.5238 
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7540 - loss: 0.5068 
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7814 - loss: 0.4680 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7813 - loss: 0.4751 
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [33]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded)

# Display the model loss and accuracy results
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7916 - loss: 0.5029  
Loss: 0.550419807434082
Accuracy: 0.7524999976158142


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [34]:
# Set the model's file path
model_path = "/content/student_loans.keras"


# Export your model to a keras file
model.save(model_path)

In [36]:
from google.colab import files
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [37]:
# Set the model's file path
model_path = "/content/student_loans.keras"

# Load the model to a new object
from tensorflow.keras.models import load_model
loaded_model = load_model(model_path)
loaded_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 22)                  │             264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 11)                  │             253 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │              24 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,625 (6.35 KB)

 Trainable params: 541 (2.11 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,084 (4.24 KB)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [38]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)


# Display a sample of the predictions
predictions[:5]

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


array([[0.73597825, 0.26402184],
       [0.44981366, 0.5501864 ],
       [0.09282617, 0.9071738 ],
       [0.20236091, 0.797639  ],
       [0.00993988, 0.9900601 ]], dtype=float32)

In [40]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions.round())

# Display the sample predictions
predictions_df

,0,1
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
395,0.0,1.0
396,1.0,0.0
397,0.0,1.0
398,1.0,0.0


### Step 4: Display a classification report with the y test data and predictions

In [42]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test_encoded, predictions_df))


              precision    recall  f1-score   support

           0       0.73      0.75      0.74       188
           1       0.77      0.75      0.76       212

   micro avg       0.75      0.75      0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400
 samples avg       0.75      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

*Starting salaries for different majors, job placement rates for different majors, average student debt held for different majors. Also the students' GPAs. These factors will determine a grad's ability to get a job, earn money, and therefore pay back the loan.*


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

*My model would lean more on Content-Based Filtering, because it would match students to loan options based on financial attributes (salary potential, debt burden, and job placement). It would not not rely on past user preferences (which would be collaborative filtering).
We may consider some economic context (but the core of our model is still content-driven).*


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

*Markets are dynamic, conditions for demand and supply for majors are not constant. For example, petroluem engineers will go from extremely high demand to extremely low demand depending on the highly volatile oil market. Interest rates and forgiveness programs will vary depending on which political party is dominant.*

